#Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input
from sklearn.model_selection import train_test_split
import numpy as np
import os
from skimage.io import imread, imshow
from skimage.color import rgb2hsv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Preparing the Data




In [ ]:
batch_size = 32
img_height = 40
img_width = 40
rescale = 1./255
data_dir = 'Dataset Filters/RGB/'

datagen=tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=rescale)

train_ds = datagen.flow_from_directory(
  directory = data_dir + 'Train',
  target_size=(img_height, img_width),
  class_mode='binary',
  color_mode= "rgb",
  batch_size=batch_size)

valid_ds = datagen.flow_from_directory(
  directory = data_dir + 'Valid',
  class_mode='binary',
  color_mode= "rgb",
  target_size=(img_height, img_width),
  batch_size=batch_size )

# Training

In [ ]:
model = tf.keras.models.Sequential()
model.add(Input(shape = (40,40,3)))
model.add(Conv2D(64, kernel_size=(5, 5), padding='same', activation='relu'))
model.add(Conv2D(96, kernel_size=(5, 5), activation='relu'))
model.add(Conv2D(1024, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=5e-5), loss= 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('best_model', monitor='val_accuracy', verbose=1, save_best_only=True)]

In [ ]:
model.fit(train_ds,
          validation_data=valid_ds,
          callbacks=callbacks,
          batch_size = 32,
          epochs=100)

In [ ]:
model = tf.keras.models.load_model('best_model')
model.save('model_color_filter_3.h5')